# Sensitivity Analysis for Olympic Sports Data

## 1. Load and Inspect Data

In [ ]:
# This cell loads the required libraries and Olympic sports data.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Provided data for sports and weights
sports_data = {
    "Safety and Fair Play": [
        0.135355, 0.101274, 0.124948, 0.121741, 0.131780, 0.126718, 0.000000,
        0.132590, 0.140585, 0.141657, 0.092674, 0.124948, 0.052205, 0.138018,
        0.135742, 0.131780, 0.032401, 0.140224
    ],
    "Gender Equity": [
        0.022844, 0.024307, 0.018792, 0.018792, 0.024222, 0.022180, 0.024944,
        0.024844, 0.000000, 0.020752, 0.021707, 0.024696, 0.023770, 0.019710,
        0.024705, 0.024944, 0.024944, 0.024944
    ],
    "Sustainability": [
        0.016928, 0.016041, 0.013695, 0.013695, 0.015681, 0.010133, 0.015681,
        0.016380, 0.017289, 0.013140, 0.016297, 0.015681, 0.006868, 0.013140,
        0.014437, 0.016041, 0.016380, 0.000000
    ],
    "Inclusivity": [
        0.044753, 0.045632, 0.010857, 0.009364, 0.007712, 0.005867, 0.008559,
        0.019006, 0.024039, 0.017680, 0.012861, 0.024344, 0.029006, 0.000000,
        0.019006, 0.028319, 0.012861, 0.026324
    ],
    "Relevance & Innovation": [
        0.030042, 0.029165, 0.028134, 0.015072, 0.015072, 0.015027, 0.015027,
        0.015027, 0.013890, 0.013890, 0.013890, 0.013131, 0.013131, 0.012943,
        0.012943, 0.011699, 0.011419, 0.000000
    ],
    "Popularity & Accessibility": [
        0.022685, 0.064399, 0.057559, 0.052747, 0.036003, 0.039500, 0.029647,
        0.000000, 0.034124, 0.020231, 0.048063, 0.016006, 0.057645, 0.036631,
        0.028721, 0.046132, 0.038335, 0.045861
    ],
}

sports_names = [
    "Swimming", "Athletics", "Baseball", "Softball", "Breaking", "Cricket",
    "Cycling-BMX Freestyle", "Cycling-Track", "Equastrian-Jumping",
    "Flag Football", "Football", "Gymnastics-Artistic", "Karate",
    "Lacrosse-Sixes", "Rowing-Coastal", "Sailing", "Skateboarding", "Squash"
]

sports_df = pd.DataFrame(sports_data, index=sports_names)

# Display the first few rows of the data
sports_df.head()


## 2. Normalize the Data

In [ ]:

# This cell normalizes the data for TOPSIS calculations.

# Normalize the data using the Euclidean norm
norm_sports_df = sports_df / np.sqrt((sports_df**2).sum())

# Display normalized data
norm_sports_df.head()


## 3. Calculate Initial Rankings

In [ ]:

# This cell calculates the initial TOPSIS scores and rankings.

weights = {
    "Safety and Fair Play": 0.502318,
    "Gender Equity": 0.095285,
    "Sustainability": 0.060981,
    "Inclusivity": 0.096594,
    "Relevance & Innovation": 0.072128,
    "Popularity & Accessibility": 0.172694,
}

# Convert weights to a NumPy array
weights_array = np.array(list(weights.values()))

# Calculate TOPSIS scores
scores = norm_sports_df @ weights_array

# Rank the sports based on scores
sports_df["TOPSIS Score"] = scores
sports_df["Rank"] = scores.rank(ascending=False)

# Display the sports with their rankings
sports_df.sort_values("Rank", inplace=True)
sports_df


## 4. Perform Sensitivity Analysis

In [ ]:

# This cell performs sensitivity analysis by adjusting weights slightly (+/- 10%).

adjustments = [-0.1, 0.1]
sensitivity_results = []

for factor, weight in weights.items():
    for adj in adjustments:
        # Adjust the current factor's weight
        adjusted_weights = weights.copy()
        adjusted_weights[factor] = weight * (1 + adj)
        
        # Normalize weights to ensure their sum equals 1
        total_weight = sum(adjusted_weights.values())
        normalized_weights = {k: v / total_weight for k, v in adjusted_weights.items()}
        
        # Recalculate TOPSIS scores with adjusted weights
        adjusted_weights_array = np.array(list(normalized_weights.values()))
        adjusted_scores = norm_sports_df @ adjusted_weights_array
        
        # Rank the sports based on adjusted scores
        adjusted_rankings = adjusted_scores.argsort()[::-1] + 1
        
        # Store the results for analysis
        sensitivity_results.append({
            "Factor": factor,
            "Adjustment": adj,
            "Adjusted Weights": normalized_weights,
            "Adjusted Rankings": adjusted_rankings,
        })

# Convert results into a DataFrame for easier analysis
sensitivity_df = pd.DataFrame(sensitivity_results)
sensitivity_df


## 5. Heatmap

In [ ]:
# This cell generates a heatmap to visualize the impact of weight adjustments on rankings.

import seaborn as sns
import matplotlib.pyplot as plt

# Extract rankings for each factor adjustment
ranking_matrix = []
factors = list(weights.keys())
for result in sensitivity_results:
    rankings = result["Adjusted Rankings"]
    ranking_matrix.append(rankings)

# Convert the ranking matrix to a DataFrame for visualization
ranking_df = pd.DataFrame(
    ranking_matrix, 
    columns=sports_names, 
    index=[f"{res['Factor']} ({res['Adjustment']:+.1f})" for res in sensitivity_results]
)

# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(ranking_df, annot=False, cmap="coolwarm", cbar_kws={'label': 'Ranking'}, linewidths=.5)
plt.title("Sensitivity Analysis Heatmap: Impact of Weight Adjustments on Rankings", fontsize=14)
plt.xlabel("Sports")
plt.ylabel("Weight Adjustments")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
